In [ ]:
# !pip3.9 install psycopg2-binary --force-reinstall --no-cache-dir

In [ ]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()

In [ ]:
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
MART_SCHEMA = os.getenv("MART_SCHEMA")

In [ ]:
def get_engine_connection():
    """
    Connects to postgreSQL DBMS on AWS Aurora
    """
    conn_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

    return create_engine(conn_string)

get_engine_connection()